In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [2]:
X_test.head(3)

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,7.036752,183.521107,20461.252710,7.333212,333.119476,356.369022,20.179029,67.019903,4.886634
1,6.643159,188.913541,32873.820022,6.791509,333.848842,336.561501,14.706810,67.844849,4.562198
2,7.846058,224.058877,23264.109968,5.922367,300.402620,387.971336,13.406737,43.075186,2.487969


In [3]:
import urllib.request
import json      


def get_prediction(x):
    ph, Hardness, Solids, Chloramines, Sulfate, Conductivity, Organic_carbon, Trihalomethanes, Turbidity = x
    body = {'ph': ph, 
            'Hardness': Hardness,
            'Solids': Solids,
            'Chloramines': Chloramines,
            'Sulfate': Sulfate,
            'Conductivity': Conductivity,
            'Organic_carbon': Organic_carbon,
            'Trihalomethanes': Trihalomethanes,
            'Turbidity': Turbidity            
            } 

    myurl = "http://127.0.0.1:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [4]:
get_prediction((1,1,1,1, 1,1,1,1,1))

0.6125735725650931

In [5]:
%%time
predictions = X_test[['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
                      'Organic_carbon', 'Trihalomethanes', 'Turbidity']].iloc[:50].apply(lambda x: get_prediction(x), 1)

CPU times: total: 125 ms
Wall time: 3.03 s


In [6]:
predictions

0     0.378211
1     0.407732
2     0.397690
3     0.411362
4     0.425370
5     0.433751
6     0.378900
7     0.398248
8     0.418447
9     0.388675
10    0.388139
11    0.377384
12    0.329672
13    0.387090
14    0.441052
15    0.435928
16    0.403073
17    0.416983
18    0.395173
19    0.408448
20    0.309404
21    0.376525
22    0.356187
23    0.403872
24    0.389628
25    0.383714
26    0.377001
27    0.350791
28    0.410914
29    0.386571
30    0.363571
31    0.397974
32    0.378237
33    0.386354
34    0.376829
35    0.382383
36    0.399972
37    0.358006
38    0.393843
39    0.427985
40    0.371957
41    0.407113
42    0.344778
43    0.399163
44    0.404309
45    0.403071
46    0.387294
47    0.404960
48    0.404282
49    0.395606
dtype: float64

In [7]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:50])

0.625